In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import numpy as np
import re
from numpy import dot
from numpy.linalg import norm
import math
import random

stop_words = set(stopwords.words('english')) 

In [2]:
data_file = "/efs/CONSORT2/data/snorkel_data/all_data_with_similar_sentences_BioBERT_embeddings_05242021.csv"
similarities_df = pd.read_csv(data_file, engine="python",encoding='utf-8', error_bad_lines=False)

In [3]:
display(similarities_df.head(5))
print(len(similarities_df))

,PMID,sentence,sid,span,section,token,lemma,pos,BERT_embeddings,core_sentence_index,core_sentence_similarity_score
0,33062915,Materials and Methods,6,"[386, 406]","['Materials and Methods', 'abstract']","['Materials', 'and', 'Methods']","['material', 'and', 'method']","['NNS', 'CC', 'NNS']",[ 1.97999671e-01 2.82724261e-01 1.07584015e-...,"[118, 234, 108, 85, 48, 100, 221, 135, 209, 14...","[0.19036911, 0.19062589, 0.19230177, 0.1937443..."
1,33062915,This study was performed as a randomized clini...,7,"[408, 635]","['Materials and Methods', 'abstract']","['This', 'study', 'was', 'performed', 'as', 'a...","['this', 'study', 'be', 'perform', 'as', 'a', ...","['DT', 'NN', 'VBD', 'VBN', 'IN', 'DT', 'JJ', '...",[ 1.00578509e-01 -1.22651041e-01 -4.63331968e-...,"[52, 219, 45, 259, 156, 146, 37, 59, 124, 28, ...","[0.2809816, 0.2842805, 0.28474694, 0.2856199, ..."
2,33062915,The primary outcome was clinical pregnancy rat...,8,"[637, 787]","['Materials and Methods', 'abstract']","['The', 'primary', 'outcome', 'was', 'clinical...","['the', 'primary', 'outcome', 'be', 'clinical'...","['DT', 'JJ', 'NN', 'VBD', 'JJ', 'NN', 'NN', 'C...",[ 1.04531892e-01 3.84195030e-01 -1.20571986e-...,"[118, 259, 241, 24, 169, 202, 199, 139, 36, 84...","[0.3363215, 0.33752206, 0.3375873, 0.3387967, ..."
3,33062915,2.,40,"[5172, 5173]",['2. Materials and Methods'],"['2', '.']","['2', '.']","['LS', '.']",[ 9.44459736e-02 3.56445044e-01 1.35182217e-...,"[131, 153, 37, 210, 66, 141, 7, 90, 253, 236, ...","[0.10177166, 0.102714114, 0.10391656, 0.103944..."
4,33062915,Materials and Methods,41,"[5175, 5195]",['2. Materials and Methods'],"['Materials', 'and', 'Methods']","['material', 'and', 'method']","['NNS', 'CC', 'NNS']",[ 1.97999671e-01 2.82724261e-01 1.07584015e-...,"[118, 234, 108, 85, 48, 100, 221, 135, 209, 14...","[0.19036911, 0.19062589, 0.19230177, 0.1937443..."


721948


In [4]:
core_sentences = "/efs/CONSORT/skr-consort/datasets/TrainingDatasets/95.txt"
core_sentences_label_0 = "/efs/CONSORT/HeuristicsBasedAnnotations/label0_200.txt"

core_sentences_lst = []
core_labels_lst = []
f = open(core_sentences, "r")
for line in f.readlines():
    sentence_items = line.split("|")
    CONSORT_Item = sentence_items[0]
    sentence_text = sentence_items[2]
    sentence_text = sentence_text.replace("\n","")
    sentence_text = sentence_text.lower()
    
    core_sentences_lst.append(sentence_text)
    core_labels_lst.append(CONSORT_Item)
    
f_0 = open(core_sentences_label_0,"r")
for line in f_0.readlines():
    sentence_items = line.split("|")
    CONSORT_Item = "0"
    sentence_text = sentence_items[3]
    sentence_text = sentence_text.replace("\n","")
    sentence_text = sentence_text.lower()
    
    core_sentences_lst.append(sentence_text)
    core_labels_lst.append(CONSORT_Item)
    
    
core_sentences_df = pd.DataFrame(
    {'CONSORT_Item': core_labels_lst,
     'text': core_sentences_lst
    })

display(core_sentences_df)

unique_labels = set(core_labels_lst)
print (unique_labels)

,CONSORT_Item,text
0,3a,"this was a multicenter, stratified (6 to 11 ye..."
1,3a,"in 6 tertiary neonatal intensive care units, w..."
2,3b,"during the trial, this committee recommended t..."
3,3b,the protocol was immediately amended in accord...
4,4a,eligible participants were all adults aged 18 ...
...,...,...
260,0,"shoe widths are measured 3a, 2a, a."
261,0,"psqi scores range from 0 to 21, with higher sc..."
262,0,this is followed by heating on a boiling water...
263,0,the application uses bluetooth to connect with...


{'11a', '12a', '3b', '10', '4a', '12b', '4b', '0', '6b', '3a', '8a', '7a', '6a', '5', '9', '7b', '8b', '11b'}


In [5]:
from collections import OrderedDict
from collections import Counter

def str_to_float_list(item):
    item = item.replace("[","")
    item = item.replace("]","")
    item = item.split(", ")
    item = [float(i) for i in item]
    return item

def str_to_int_list(item):
    item = item.replace("[","")
    item = item.replace("]","")
    item = item.split(", ")
    item = [int(i) for i in item]
    return item

count = 0
for index, row in similarities_df.iterrows():
    if count%500 == 0:
        print (count)
    row_similarity_score = row["core_sentence_similarity_score"]
    row_similarity_score = str_to_float_list(row_similarity_score)

    row_similarity_index = row["core_sentence_index"]
    row_similarity_index = str_to_int_list(row_similarity_index)

    #get top 10 similar core sentences
    top_n_similarity_indexes = row_similarity_index[85:]
    top_n_similar_core_sentences_df = core_sentences_df[core_sentences_df.index.isin(top_n_similarity_indexes)]  
    top_n_similar_core_sentences_df = top_n_similar_core_sentences_df.reindex(top_n_similarity_indexes)
    top_n_CONSORT_Items = top_n_similar_core_sentences_df["CONSORT_Item"].to_list()

    top_n_CONSORT_Items_by_frequency = Counter(top_n_CONSORT_Items)
    top_n_CONSORT_Items_by_frequency = OrderedDict(sorted(top_n_CONSORT_Items_by_frequency.items(), key = lambda kv : kv[1], reverse=True))

    highest_frequency_label = top_n_CONSORT_Items_by_frequency.popitem(last=False)[0]
    highest_similarity_label = top_n_CONSORT_Items[9]

    
    similarities_df.at[index, "highest_frequency_label_in_top10"] = highest_frequency_label
    similarities_df.at[index, "highest_similarity_label"] = highest_similarity_label
    
    count +=1
    
display(similarities_df.head(10))

similarities_df.to_csv("/efs/CONSORT2/data/snorkel_data/all_data_with_Sentences_Similarity_Info_05262021.csv")


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500
55000
55500
56000
56500
57000
57500
58000
58500
59000
59500
60000
60500
61000
61500
62000
62500
63000
63500
64000
64500
65000
65500
66000
66500
67000
67500
68000
68500
69000
69500
70000
70500
71000
71500
72000
72500
73000
73500
74000
74500
75000
75500
76000
76500
77000
77500
78000
78500
79000
79500
80000
80500
81000
81500
82000
82500
83000
83500
84000
84500
8500

601500
602000
602500
603000
603500
604000
604500
605000
605500
606000
606500
607000
607500
608000
608500
609000
609500
610000
610500
611000
611500
612000
612500
613000
613500
614000
614500
615000
615500
616000
616500
617000
617500
618000
618500
619000
619500
620000
620500
621000
621500
622000
622500
623000
623500
624000
624500
625000
625500
626000
626500
627000
627500
628000
628500
629000
629500
630000
630500
631000
631500
632000
632500
633000
633500
634000
634500
635000
635500
636000
636500
637000
637500
638000
638500
639000
639500
640000
640500
641000
641500
642000
642500
643000
643500
644000
644500
645000
645500
646000
646500
647000
647500
648000
648500
649000
649500
650000
650500
651000
651500
652000
652500
653000
653500
654000
654500
655000
655500
656000
656500
657000
657500
658000
658500
659000
659500
660000
660500
661000
661500
662000
662500
663000
663500
664000
664500
665000
665500
666000
666500
667000
667500
668000
668500
669000
669500
670000
670500
671000
671500
672000
672500

,PMID,sentence,sid,span,section,token,lemma,pos,BERT_embeddings,core_sentence_index,core_sentence_similarity_score,highest_frequency_label_in_top10,highest_similarity_label
0,33062915,Materials and Methods,6,"[386, 406]","['Materials and Methods', 'abstract']","['Materials', 'and', 'Methods']","['material', 'and', 'method']","['NNS', 'CC', 'NNS']",[ 1.97999671e-01 2.82724261e-01 1.07584015e-...,"[118, 234, 108, 85, 48, 100, 221, 135, 209, 14...","[0.19036911, 0.19062589, 0.19230177, 0.1937443...",0,0
1,33062915,This study was performed as a randomized clini...,7,"[408, 635]","['Materials and Methods', 'abstract']","['This', 'study', 'was', 'performed', 'as', 'a...","['this', 'study', 'be', 'perform', 'as', 'a', ...","['DT', 'NN', 'VBD', 'VBN', 'IN', 'DT', 'JJ', '...",[ 1.00578509e-01 -1.22651041e-01 -4.63331968e-...,"[52, 219, 45, 259, 156, 146, 37, 59, 124, 28, ...","[0.2809816, 0.2842805, 0.28474694, 0.2856199, ...",0,3a
2,33062915,The primary outcome was clinical pregnancy rat...,8,"[637, 787]","['Materials and Methods', 'abstract']","['The', 'primary', 'outcome', 'was', 'clinical...","['the', 'primary', 'outcome', 'be', 'clinical'...","['DT', 'JJ', 'NN', 'VBD', 'JJ', 'NN', 'NN', 'C...",[ 1.04531892e-01 3.84195030e-01 -1.20571986e-...,"[118, 259, 241, 24, 169, 202, 199, 139, 36, 84...","[0.3363215, 0.33752206, 0.3375873, 0.3387967, ...",12a,6a
3,33062915,2.,40,"[5172, 5173]",['2. Materials and Methods'],"['2', '.']","['2', '.']","['LS', '.']",[ 9.44459736e-02 3.56445044e-01 1.35182217e-...,"[131, 153, 37, 210, 66, 141, 7, 90, 253, 236, ...","[0.10177166, 0.102714114, 0.10391656, 0.103944...",0,0
4,33062915,Materials and Methods,41,"[5175, 5195]",['2. Materials and Methods'],"['Materials', 'and', 'Methods']","['material', 'and', 'method']","['NNS', 'CC', 'NNS']",[ 1.97999671e-01 2.82724261e-01 1.07584015e-...,"[118, 234, 108, 85, 48, 100, 221, 135, 209, 14...","[0.19036911, 0.19062589, 0.19230177, 0.1937443...",0,0
5,33062915,This study was a randomized clinical trial per...,42,"[5197, 5374]",['2. Materials and Methods'],"['This', 'study', 'was', 'a', 'randomized', 'c...","['this', 'study', 'be', 'a', 'randomized', 'cl...","['DT', 'NN', 'VBD', 'DT', 'JJ', 'JJ', 'NN', 'V...",[-6.55009449e-02 -2.85554916e-01 -1.07530713e-...,"[227, 159, 11, 125, 73, 104, 19, 4, 188, 156, ...","[0.24631405, 0.24675846, 0.25034636, 0.2523195...",0,4b
6,33062915,"The inclusion criteria were age <40 yr, at lea...",43,"[5376, 5672]",['2. Materials and Methods'],"['The', 'inclusion', 'criteria', 'were', 'age'...","['the', 'inclusion', 'criterion', 'be', 'age',...","['DT', 'NN', 'NNS', 'VBD', 'NN', 'SYM', 'CD', ...",[-1.62878856e-01 2.65930086e-01 -1.77194327e-...,"[251, 1, 249, 35, 93, 21, 219, 23, 45, 82, 118...","[0.33169034, 0.3326995, 0.33507633, 0.33514705...",4a,4a
7,33062915,128 eligible women who were assigned for FET p...,44,"[5674, 5744]",['2. Materials and Methods'],"['128', 'eligible', 'women', 'who', 'were', 'a...","['128', 'eligible', 'woman', 'who', 'be', 'ass...","['CD', 'JJ', 'NNS', 'WP', 'VBD', 'VBN', 'IN', ...",[ 2.31100827e-01 1.53269935e-02 -3.23140800e-...,"[90, 107, 253, 123, 91, 17, 160, 256, 25, 110,...","[0.19466507, 0.19504952, 0.19650891, 0.1970538...",0,0
8,33062915,They were allocated into two groups randomly a...,45,"[5746, 5802]",['2. Materials and Methods'],"['They', 'were', 'allocated', 'into', 'two', '...","['they', 'be', 'allocate', 'into', 'two', 'gro...","['PRP', 'VBD', 'VBN', 'IN', 'CD', 'NNS', 'RB',...",[ 1.49566233e-01 -3.12592655e-01 -9.73997042e-...,"[67, 187, 252, 2, 1, 199, 247, 9, 94, 107, 33,...","[0.22436488, 0.22610599, 0.22645034, 0.2268338...",10,3b
9,33062915,"Group A (n = 64) was prescribed 80 mg aspirin,...",46,"[5804, 5943]",['2. Materials and Methods'],"['Group', 'A', '(', 'n', '=', '64', ')', 'was'...","['Group', 'A', '(', 'n', '=', '64', ')', 'be',...","['NNP', 'NNP', '-LRB-', 'NN', 'SYM', 'CD', '-R...",[-6.18886836e-02 5.21771517e-03 -1.97594538e-...,"[53, 235, 146, 84, 90, 58, 25, 163, 237,

In [10]:
# Merge with the actual data for snorkel
snorkel_data_file = "/efs/CONSORT2/data/snorkel_data/all_methodological_data_preprocessed_05232021.csv"

snorkel_data_df = pd.read_csv(snorkel_data_file)
display(snorkel_data_df.head(5))
print (len(snorkel_data_df))

,index,PMID,sid,text,section_hierarchy
0,0,33062915,7,this study was performed as a randomized clini...,"['materials and methods', 'abstract']"
1,1,33062915,8,the primary outcome was clinical pregnancy rat...,"['materials and methods', 'abstract']"
2,2,33062915,42,this study was a randomized clinical trial per...,['2. materials and methods']
3,3,33062915,43,"the inclusion criteria were age <40 yr, at lea...",['2. materials and methods']
4,4,33062915,44,128 eligible women who were assigned for fet p...,['2. materials and methods']


599166


In [11]:
merged_snorkel_data_df = snorkel_data_df.merge(similarities_df, on = ["PMID", "sid"], how = "left")
print (len(merged_snorkel_data_df))

599166


In [14]:
merged_snorkel_data_df = merged_snorkel_data_df[["PMID", "sid","text","section_hierarchy","highest_frequency_label_in_top10", "highest_similarity_label"]]
merged_snorkel_data_df = merged_snorkel_data_df.reset_index()
display(merged_snorkel_data_df.head(5))
merged_snorkel_data_df.to_csv("/efs/CONSORT2/data/snorkel_data/all_methodological_data_preprocessed_withSentenceSimilarityInfo_05262021.csv", index = False)


,index,PMID,sid,text,section_hierarchy,highest_frequency_label_in_top10,highest_similarity_label
0,0,33062915,7,this study was performed as a randomized clini...,"['materials and methods', 'abstract']",0,3a
1,1,33062915,8,the primary outcome was clinical pregnancy rat...,"['materials and methods', 'abstract']",12a,6a
2,2,33062915,42,this study was a randomized clinical trial per...,['2. materials and methods'],0,4b
3,3,33062915,43,"the inclusion criteria were age <40 yr, at lea...",['2. materials and methods'],4a,4a
4,4,33062915,44,128 eligible women who were assigned for fet p...,['2. materials and methods'],0,0
